In [1]:
import os

import geemap
import ee
import eelib
from dotenv import load_dotenv
from pipelines import eerfpl, factories, colors

load_dotenv()

True

In [2]:
viewport = ee.FeatureCollection.from_file(
    filename = "../000-data/viewport.shp"
).geometry()

training_data = ee.FeatureCollection.from_file(
    filename="../../phase01/000-data/training_data.gdb",
    driver='FileGDB',
    layer='training_points_700'
)


In [3]:
with open("./systems.txt", 'r') as file:
    ids = [line.strip() for line in file.readlines()]
ids

['S1B_IW_GRDH_1SDV_20180609T015452_20180609T015517_011290_014BA0_39FD',
 'S1B_IW_GRDH_1SDV_20180609T015517_20180609T015542_011290_014BA0_2658',
 'S1B_IW_GRDH_1SDV_20180609T015542_20180609T015616_011290_014BA0_EE30',
 'S1B_IW_GRDH_1SDV_20180715T015454_20180715T015519_011815_015BE0_0362',
 'S1B_IW_GRDH_1SDV_20180715T015519_20180715T015544_011815_015BE0_D8D7',
 'S1B_IW_GRDH_1SDV_20180715T015544_20180715T015618_011815_015BE0_8287',
 'S1B_IW_GRDH_1SDV_20180913T015503_20180913T015528_012690_0176B4_78B3',
 'S1B_IW_GRDH_1SDV_20180913T015528_20180913T015553_012690_0176B4_0EB4',
 'S1B_IW_GRDH_1SDV_20180913T015553_20180913T015613_012690_0176B4_EB44']

In [4]:
ee_images = [ee.Image(f'COPERNICUS/S1_GRD/{_}') for _ in ids]
ee_images

In [5]:
# img_col = ee.ImageCollection(ee_images).map(lambda x: x.set('dateFmt', x.date().format('YYYY-MM-dd')))
# dates = img_col.aggregate_array('dateFmt').distinct()
# dates_list = dates.getInfo()

# s1_inputs = [
#     img_col.filter(f'dateFmt == "{date}"').mosaic() for date in dates_list
# ]
# s1_inputs = eefuncs.batch_despeckle(s1_inputs, sf.Boxcar(1))

datacube = factories.DatacubeCollection(
    arg="projects/fpca-336015/assets/williston-cba"
).filterBounds(viewport)

s1_collection = ee.ImageCollection(ee_images)

# datacube_inputs = factories.datacube_img_factory(
#     datacube_collection=datacube,
#     viewport=viewport
# )

dem = ee.Image("NASA/NASADEM_HGT/001").select('elevation')

In [6]:
pipeline = eerfpl.DataCubePipelineLSC(
    sar=s1_collection,
    optical=datacube,
    dem=dem,
    training_data=training_data
)

# Export Settings
pipeline.bucket = os.environ.get('BUCKET')
pipeline.root = os.environ.get('ROOT')
pipeline.caseNumber = "Phase02"
pipeline.region = viewport

output = pipeline.run()

pipeline.logging()




In [7]:
Map = geemap.Map()

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
vis = {'palette': [f'#{_.value}' for _ in colors.Colors], 'min': 1, 'max': 7}
Map.centerObject(viewport, 6)

Map.addLayer(output.get('image'), vis, 'LSC Williston')


In [10]:
pipeline.exportToCloud()